In [2]:
from __future__ import annotations
from typing import Any
from pydantic import BaseModel, Field
from pydantic._internal._model_construction import ModelMetaclass
from typing import Dict
from uuid import uuid4

class SignalDescriptor:
    """Return `$Model.field` on the class, real value on an instance."""

    def __init__(self, field_name: str) -> None:
        self.field_name = field_name

    def __get__(self, instance, owner):
        #  class access  →  owner is the model class, instance is None
        if instance is None:
            config = getattr(owner, "model_config", {})
            ns = config.get("namespace", owner.__name__)
            use_ns = config.get("use_namespace", False)
            return f"${ns}.{self.field_name}" if use_ns else f"${self.field_name}"
        return instance.__dict__[self.field_name]        

        #  instance access  →  behave like a normal attribute

class SignalModelMeta(ModelMetaclass):
    def __init__(cls, name, bases, ns, **kw):
        super().__init__(name, bases, ns, **kw)

        for field_name in cls.model_fields:
            setattr(cls, f"{field_name}_signal", SignalDescriptor(field_name))
        for field_name in cls.model_computed_fields:
            setattr(cls, f"{field_name}_signal", SignalDescriptor(field_name))

class User(BaseModel, metaclass=SignalModelMeta):
    # id: str = Field(default_factory=lambda: str(uuid4()), primary_key=True)
    id: str

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # self.id = 123

    model_config = {
        "namespace": "User",
        "use_namespace": True,
    }

    @property
    def signals(self) -> Dict[str, Any]:
        return self.model_dump()
    
    @property
    def signals_ns(self) -> Dict[str, Any]:
        "namespaced signals"
        return {self.__class__.__name__:self.signals}
    
    @property
    def scope(self) -> str:
        "scope as string"
        return self.__class__._get_config_value("scope", "Unnamed")
    
    @classmethod
    def _get_config_value(cls, key: str, default=None):
        """Get configuration value from model_config."""
        return cls.model_config.get(f"stm_{key}", default)
    
    def signal(self, field: str) -> Any:
        if field in self.signals.keys():
            return f"${self.__class__.__name__}.{field}"
        else:
            raise ValueError(f"Field {field} not found in {self.__class__.__name__}")


f = User.model_fields['id'].get_default(call_default_factory=True)


In [3]:
from pydantic import computed_field
from pydantic.dataclasses import dataclass

class User(BaseModel, metaclass=SignalModelMeta):
    id: str = Field(default_factory=lambda: str(uuid4()), primary_key=True)
    name: str
    age: int

class Person(BaseModel, metaclass=SignalModelMeta):
    model_config = {
        "namespace": "Person"
    }
    name: str
    lastName: str

    @computed_field
    def full_name(self) -> str:
        return f"{self.name} {self.lastName}"
    

person = Person(name="John", lastName="Doe")
Person.model_fields
Person.full_name_signal

'$full_name'

In [4]:
Person.full_name_signal

'$full_name'

In [5]:
from pydantic import BaseModel, Field
from pydantic.dataclasses import dataclass
from typing import get_type_hints, get_origin, get_args
import copy

def model2dataclass(model_cls: type[BaseModel]) -> type[dataclass]:
    """Convert BaseModel to dataclass preserving validators and complex types"""
    
    # Get model configuration
    config = getattr(model_cls, 'model_config', {})
    
    # Extract field information
    fields = model_cls.model_fields
    annotations = get_type_hints(model_cls)
    
    # Prepare class attributes
    class_attrs = {'__annotations__': annotations.copy()}
    
    # Process each field
    for field_name, field_info in fields.items():
        if field_info.default is not ...:
            class_attrs[field_name] = field_info.default
        elif field_info.default_factory is not None:
            class_attrs[field_name] = Field(default_factory=field_info.default_factory)
        
        # Preserve field constraints and metadata
        if hasattr(field_info, 'constraints') or hasattr(field_info, 'metadata'):
            field_copy = copy.deepcopy(field_info)
            class_attrs[field_name] = field_copy
    
    # Create the dataclass with config
    DataClass = type(f"{model_cls.__name__}DataClass", (), class_attrs)
    
    # Apply dataclass decorator with config
    return dataclass(DataClass, config=config)

In [6]:
UserDS = model2dataclass(User)

user1_ds = UserDS(name="John", age=30)
user1_ds.name
type(user1_ds)

BaseModel.__subclasses__()

ds_classes = {ds.__name__: model2dataclass(ds) for ds in BaseModel.__subclasses__()}
ds_classes

{'SQLModel': __main__.SQLModelDataClass,
 'User': __main__.UserDataClass,
 'Person': __main__.PersonDataClass}

In [7]:
from pydantic import BaseModel, ConfigDict
from sqlmodel import SQLModel, Field
from typing import Optional
from datetime import datetime
from FastSQLModel.db import BaseTable

class EntityConfig(ConfigDict):
    """Configuration for all entity classes."""
    namespace: str | None
    use_namespace: bool
    auto_persist: bool
    persistence_backend: str
    sync_with_client: bool
    ttl: Optional[int]

class Entity(BaseModel):
    """Base class for all entity classes."""
    model_config = EntityConfig(arbitrary_types_allowed=True,
                                use_namespace=True,
                                auto_persist=True,
                                persistence_backend="MemoryRepo()",
                                sync_with_client=True,
                                json_encoders={datetime: lambda dt: dt.isoformat()})
    
    @classmethod
    def __pydantic_init_subclass__(cls, **kwargs: Any) -> None:
        super().__pydantic_init_subclass__(**kwargs)
        
        # Create signal descriptors for all model fields
        for field_name in cls.model_fields:
            setattr(cls, f"{field_name}_signal", SignalDescriptor(field_name))
        for field_name in cls.model_computed_fields:
            setattr(cls, f"{field_name}_signal", SignalDescriptor(field_name))

# Step 1: Define your existing Pydantic model
class UserBase(Entity, BaseTable, table=True):
    __table_args__ = {'extend_existing': True}
    name: str
    email: str

user = UserBase(name="John", email="john@example.com")
user.model_dump_json()

'{"name":"John","email":"john@example.com","id":"35aa7b58-3c81-4085-a8cf-d5a956b4867d","created_at":"2025-06-20T09:49:04.797230+00:00","updated_at":"2025-06-20T09:49:04.797243+00:00"}'

In [61]:
from abc import ABC, abstractmethod
from pydantic import BaseModel, Field
import uuid
from fasthtml.common import Div, Span, Html
from IPython.display import IFrame 

class Renderable(BaseModel):
    @abstractmethod
    def render(self) -> str:
        pass
class Repo(BaseModel):
    @abstractmethod
    def save(self, data: dict) -> None:
        pass
    
    @abstractmethod
    def load(self, id: str) -> dict:
        pass

class Stringy(Renderable):
    def render(self, content: str) -> str:
        return f"Stringy: {content}"

class Htmly(Renderable):
    def render(self, content: BaseModel) -> str:
        return Div(*[Span(f"{k}: {v}") for k, v in content.model_dump().items()])

class MemoryRepo(Repo):
    data: dict = {}

    def save(self, data: dict) -> None:
        self.data[data["id"]] = data
    
    def load(self, id: str) -> dict:
        return self.data[id]

class DefaultConfig:
    arbitrary_types_allowed = True
    renderable: Renderable = Stringy()
    repo: Repo = MemoryRepo()
    extra='allow'
    
    def __init__(self, renderable: Renderable = Stringy(), repo: Repo = MemoryRepo()):
        self.renderable = renderable
        self.repo = repo



class Composable(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))

    class Config(DefaultConfig):
        pass
    
    def render(self) -> str:
        return self.Config.renderable.render(self)
    
    def save(self) -> None:
        self.Config.repo.save(self.model_dump())
    
    def load(self) -> None:
        return self.Config.repo.load(self.model_dump()["id"])
    
    def __str__(self) -> str:
        return self.render()

class Email(Composable):
    to: str
    subject: str
    body: str

    class Config(DefaultConfig):
        renderable = Htmly()

email = Email(to="nikola@test.com", subject="Test", body="Test", ps="All good here!")
# email.save()
# email.load(email.id)
email.render()

```html
<div>
<span>id: 33412279-2189-4143-b171-89425da4fffd</span><span>to: nikola@test.com</span><span>subject: Test</span><span>body: Test</span><span>ps: All good here!</span></div>

```